# Spider Movement Analysis using PCA

# Part 3: Leg Agnostic PCA

## Introduction

This notebook analyses how spiders move by looking at 3D recordings of their body positions. We use a technique called Principal Component Analysis (PCA) to break down complex spider movements into simpler, fundamental motion patterns. This helps us understand how spiders coordinate their legs and body during different behaviours.

Here we will rotate and reflect the legs such that every leg is in the same position relative to the body. This means the PCA will find patterns in the legs that are independent of leg identity. While the spider no doubtedly coordinates its legs, this analysis is about finding the controls of each leg. Comparing the scores for each leg will then give some idea of how they are used differently. 

Treating the legs agnostically has an advantage over the full shape PCA. There will be a higer number of rows to columns (as there is now just coxa, patella, tibia-metatarsal, and claw, not claws 1-8 etc). The effect of measurement error will now be much smaller. We can find the coordinations across legs by looking at the scores, but this leg agnostic model could also be the first step to remove spurious datapoints. 

The main challenge is rotating the legs to the same position. We cannot use a "leg vector" from the base of the leg to some point on the leg because this would remove the rotational freedom of the leg that is controlled by the spider. For example, if we measure a vector from the base of the leg (coxa) to the patella (first joint), for every leg position this will remove the actual rotation of the leg at this joint. The same for coxa to claw or anything else on the leg. 

Here I have tried to find a vector between each coxa and some other reference point on the cephalothorax. The cephalothorax should be rigid, but there is freedom of movement between the cephalothorax and abdomen. I have also reflected the left side of the spider to the right to copy the bilateral symmetry of the spider. 

I am also going to test whether a leg-agnostic PCA with reflection can find the leg-direction as a component. 


<img src="./images/spider_defs.png" alt="Spider" style="width: 60%;">

## Setup and Data Loading

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='svg'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from pathlib import Path

from morphing_birds import Spider3D, plot_plotly, animate_plotly, animate_plotly_compare, animate
from spiderpca import (run_PCA, plot_explained, get_score_range, reconstruct, 
                       create_scores_dataframe, plot_pc_experiment, plot_pc_histogram)